In [ ]:
## standard library
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import scipy as sp
import pandas as pd
import time
import json
import pickle

#Image processing packages
import cv2
from PIL import Image

## Keras + Tensorflow
from keras.utils.np_utils import to_categorical
from keras.models import load_model
from keras.models import Sequential
from keras.layers.core import Flatten, Dense, Dropout, Lambda, Activation
from keras.layers.convolutional import Conv2D, MaxPooling2D
from keras.optimizers import Adam, SGD
from keras.preprocessing import image
from keras.utils import np_utils
import tensorflow as tf

%matplotlib inline

In [ ]:
f = open(r'./ships-in-satellite-imagery/shipsnet_v2.json')
dataset = json.load(f)
f.close()

### Detailed description of the final test code. Slight modification for use in scan image.ipynb

In [ ]:
data = np.array(dataset['data']).astype('uint8')
labels = np.array(dataset['labels']).astype('uint8')

In [ ]:
print(data.shape)
print(labels.shape)

In [ ]:
x = data / 255.

# transpose the position according to the axis order specify in the list\
# [0,2,3,1] -- shape[0], shape[2], shape [3], shape[1]
# reshape --> -1 means keeping the dimension value at axis[0]

x = x.reshape([-1, 3, 80, 80]).transpose([0,2,3,1])

print(x.shape)

In [ ]:
y = to_categorical(labels, num_classes=2)

print(y.shape)

In [ ]:
# CNN model. Starting point from keras tutorial: VGG-like convnet
# https://keras.io/getting-started/sequential-model-guide/#training

model = Sequential()

model.add(Conv2D(32, (3, 3), padding="same", input_shape=(80, 80, 3), activation='relu'))
model.add(Conv2D(32, (3, 3), activation='relu'))

## batch normalization -- code

model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Dropout(0.2))

model.add(Conv2D(64, (3, 3), padding="same", activation='relu'))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Dropout(0.2))

# Potential Third Layer
# model.add(Conv2D(128, (3, 3), padding="same", activation='relu'))
# model.add(Conv2D(128, (3, 3), activation='relu'))
# model.add(MaxPooling2D(pool_size=(2, 2)))
# model.add(Dropout(0.2))


model.add(Flatten())
## FC layer 1
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.2))
## FC layer 2 with softmax classifer
model.add(Dense(2, activation='softmax'))

## compile a model
model.compile(loss='categorical_crossentropy', optimizer=SGD(lr=0.001, momentum=0.9, nesterov=True), metrics=['accuracy'])

In [ ]:
train_start = time.time()

history = model.fit(x, y, batch_size=64, epochs=150, validation_split=0.2)

train_end = time.time()

In [ ]:
train_time = train_end - train_start
print ('CNN model train time = {}'.format(train_time))

In [ ]:
## save train history dict and load train history dict to test.

In [ ]:
with open('./cnn_model/classiferCNN/model2_hist.pkl', 'wb') as file_name:
    pickle.dump(history.history, file_name)

In [ ]:
with open('./cnn_model/classiferCNN/model2_hist.pkl','rb') as file_name:
    modelnew_hist = pickle.load(file_name)

In [ ]:
## save model and load model to test.

In [ ]:
model.save('./cnn_model/classiferCNN/model2.h5')

In [ ]:
modelnew = load_model('./cnn_model/classiferCNN/model2.h5')

In [ ]:
## plotting the history of the CNN training

fig, ax = plt.subplots(nrows=1, ncols=2, figsize=(20,8))

ax[0].plot(modelnew_hist['acc'])
ax[0].plot(modelnew_hist['val_acc'])
ax[0].set_title('model accuracy')
ax[0].set_ylabel('accuracy')
ax[0].set_xlabel('epoch')
ax[0].legend(['train', 'test'], loc='upper left')

ax[1].plot(modelnew_hist['loss'])
ax[1].plot(modelnew_hist['val_loss'])
ax[1].set_title('model loss')
ax[1].set_ylabel('loss')
ax[1].set_xlabel('epoch')
ax[1].legend(['train', 'test'], loc='upper left')

plt.show()

In [ ]:
### scanning of the image

In [ ]:
def sliding_window(image, stepsize , winsize):
    # image is always specify by height first then width
    
    # slide a window across the satellite image
    for y in range(0, image.shape[0], stepsize[0]):
        for x in range(0, image.shape[1], stepsize[1]):
            # yield to generate the coordinates and window image for use in the classifier
            # as we scan through the image
            yield (x, y, x + winsize[1], y + winsize[0], image[y:y + winsize[0], x:x + winsize[1]])

In [ ]:
def non_max_suppression(bound_box, area_overlap_threshold):
    
    # bound_box => np.array of bounding box coordinate[(x1,y1,x2,y2),(),()] for numpy slicing
    boxes = np.array(bound_box)
    
    # if there are no boxes, return an empty list
    if len(boxes) == 0:
        return []
 
    # initialize a list for pick boxes coordinates
    pick = []
 
    # x1, y1 top left point of each bounding box
    # x2, y2 bottem right point of each bounding box
    x1 = boxes[:,0]
    y1 = boxes[:,1]
    x2 = boxes[:,2]
    y2 = boxes[:,3]
 
   
    # Default bounding box area is the same as the search window --> 80*80 == 6400
    area = 6400.0
    
    # Sort the index of the bounding boxes by y2 (bottem right)
    ind = np.argsort(y2)
 
    while len(ind) > 0:
        end = len(ind) - 1
        i = ind[end] # get the last index of the ind list for coordinates (x1,y1,x2,y2)
        pick.append(i) # pick the last bounding box in the list (bottom right most bounding box in the full sat image)
        suppress = [end] # place the index of last bounding box in suppress
        
        # At the end of each for loop, the index of last bounding box will be deleted from ind list 
        # as it is already been place in the pick list
        
        # Each loop will try to find neighbouring boxes with high overlapping area with the last bounding box in the list
        # Once found, suppress these neighbouring boxes and only keep the bottom right most
        
        for coord in range(0, end):
            j = ind[coord]
            # Loop through all the remaining bounding boxes in ind list
            
            # find the largest (x, y) coordinates for the start of the bounding box
            # find smallest (x, y) coordinates for the end of the bounding box
            
            # This is used to calculate the overlapping area
            xx1 = max(x1[i], x1[j])
            yy1 = max(y1[i], y1[j])
            xx2 = min(x2[i], x2[j])
            yy2 = min(y2[i], y2[j])
 
            # compute the width and height of the bounding box
            
            # Note that the min, max selection of coordinates is with reference to the last bounding box in the image
            # x[i] is the last bounding box with the highest x1,y1,x2,y2 value when compared to non-overlapping boxes.
            # Therefore:
            
            # xx2 - xx1 will be negative and w = 0
            # yy2 - yy1 will be positive and h = 0
            
            # if there is no overlapping of the boxes i, j.
        
            w = max(0, xx2 - xx1 + 1)
            h = max(0, yy2 - yy1 + 1)
 
            # compute the ratio of overlap
            # Bounding box area is only 1 size of area 6400
            overlap = (w * h) / area
        
            # check if the overlap ratio exceed a certain value. 
            # if so, put it into the suppress list
            if overlap > area_overlap_threshold:
                suppress.append(coord)
 
        # delete all index in suppress list from ind list.
        ind = np.delete(ind, suppress)
 
    return boxes[pick]

    

In [ ]:
satimage_scan = cv2.imread('./ships-in-satellite-imagery/test_v2/sfbay_1.png')
satimage_result = cv2.imread('./ships-in-satellite-imagery/test_v2/sfbay_1.png')
satimage_nms = cv2.imread('./ships-in-satellite-imagery/test_v2/sfbay_1.png')
print (satimage_scan.shape[0])
print (satimage_scan.shape[1])

In [ ]:
(winH, winW) = (80, 80)
(stepY,stepX) = (10, 10)

In [ ]:
# loop sliding window over the enitre satellite image

pred_class = []
pred_prob = []
win_img = []
win_coordinate = []

ship_pred_class = []
ship_pred_prob = []
ship_img = []
ship_cooridinate = []

i = 0

## time for the sliding window to scan the whole satellite image
scan_start = time.time()

for (x1, y1, x2, y2, window) in sliding_window(satimage_scan, stepsize=(stepY,stepX), winsize=(winH, winW)):
    
    # If the window does not meet our desired window size, continue 
    # For windows reaching the edge, ignoring those windows not the same size as the specified winsize)
    
    if window.shape[0] != winH or window.shape[1] != winW:
        continue

    # Classifier Code
    # Classify content within the sliding window
    # input a new dims on the window so that it can be input into the CNN classifier (take in only a tensor)
    target_region = np.expand_dims(window, axis=0)
    classes = modelnew.predict_classes(target_region, train=False)
    x_proba = modelnew.predict(target_region)
   
    pred_class.append(classes[0])
    pred_prob.append(x_proba)
    win_img.append(window)
    win_coordinate.append((x1,y1,x2,y2))
    
    # classes [0] = 1 is ship
    # classes [0] = 0 is not ship
    
    if classes[0] == 1:
        
        cv2.imwrite("./results/classiferCNN/winimg2/rimg_{}.png".format(i), window)
        
        ship_pred_class.append(classes[0])
        ship_pred_prob.append(x_proba)
        ship_img.append(window)
        ship_cooridinate.append((x1,y1,x2,y2))
        
        # cv2.namedWindow("Window", cv2.WINDOW_NORMAL) 
        # draw a rectange green box at the pixel coordinate (x, y) and (x + winW, y + winH) with box line thickness of 2 pixel
        cv2.rectangle(satimage_result, (x1, y1), (x1 + winW, y1 + winH), (0, 255, 0), 2) 
        #cv2.imshow("Windows",satimage)
        
        # Save the big satellite image for gif generation
        cv2.imwrite("./results/classiferCNN/gif2/img_{}.png".format(i), satimage_result)
        
        # Save the window image that has a bounding box
        
        i += 1
        
        #cv2.waitKey(1)
        #time.sleep(0.025)
        
        ### save png files
    #else:
        #cv2.waitKey(1)
        #time.sleep(0.025)


# Final result of satimage_result
# Everytime satimage_result will update with more green boxes
cv2.imwrite("./results/classiferCNN/final_result/detected2.png", satimage_result)
cv2.destroyAllWindows()
                          
scan_end = time.time()
scan_time = scan_end - scan_start

print(scan_time)

print('-------------------')

In [ ]:
result_data = {'pred_class': pred_class, 'pred_prob': pred_prob, 'win_img': win_img, 'win_coordinate': win_coordinate}
predict_ship = {'ship_pred_class': ship_pred_class, 'ship_pred_prob': ship_pred_prob, 'ship_img': ship_img, 'ship_cooridinate': ship_cooridinate}

with open('./results/classiferCNN/result_data2.pkl', 'wb') as file_name:
    pickle.dump(result_data, file_name)
    
with open('./results/classiferCNN/predict_ship2.pkl', 'wb') as file_name:
    pickle.dump(predict_ship, file_name)

In [ ]:
## NMS to suppress overlapping bounding boxes
# area overlap ratio = 0.5, suppress boxes that overlap more then half the area

print('NMS')
satimage_nms = cv2.imread('./ships-in-satellite-imagery/test_v2/sfbay_1.png')
picked_box = non_max_suppression(ship_cooridinate, area_overlap_threshold = 0.1)

for (bbx1, bby1, bbx2, bby2) in picked_box:
    cv2.rectangle(satimage_nms, (bbx1, bby1), (bbx2,bby2), (0, 0, 255), 2) 

        
cv2.imwrite("./results/classiferCNN/detected2_NMS.png", satimage_nms)

print('Save NMS results')

In [ ]:
cv2.destroyAllWindows()